In [1]:
import re
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.client import device_lib
from tensorflow.keras.models import load_model
from eunjeon import Mecab
import numpy as np
import pickle

In [2]:
# 토크나이저 로드 함수
def load_tokenizer(filepath):
    with open(filepath, 'rb') as f:
        tokenizer = pickle.load(f)
    return tokenizer

In [3]:
from konlpy.tag import Mecab

In [4]:
data = pd.read_csv('./data/tokenized_data.csv', encoding_errors='ignore', low_memory=False)
data['new_label']=None
data['tokenized_comment']=None
loaded_model = load_model('./model/final_model_v0.h5')
tokenizer = Tokenizer()
mecab = Mecab('C:/mecab/mecab-ko-dic')

In [5]:
# 예측 과정에서의 토크나이저 로드
tokenizer = load_tokenizer('./model/tokenizer.pkl')
stopwords = load_tokenizer('./model/stopwords.pkl')

In [6]:
def sentiment_predict(new_sentence):
    new_sentence = mecab.morphs(new_sentence)
    new_sentence = [word for word in new_sentence if word not in stopwords]
    
    if not new_sentence:
        print("토큰이 없어 중립 리뷰입니다.")
        return -1
    
    # 시퀀스 길이 설정
    max_len = 80
    
    # 시퀀스를 인코딩하고 패딩
    encoded = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(encoded, maxlen=max_len)
    
    # 모델에 입력하고 예측 수행
    score = loaded_model.predict(pad_new)[0]
    
    score = float(score)  # 배열에서 스칼라 값으로 변환
    
    if score > 0.5:
        print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
        return 1
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))
        return 0


In [7]:
data['tokenized_comments'] = None

In [9]:
for index, row in data.iterrows():
    sp = sentiment_predict(row['Review'])
    if sp == 1:
        data.at[index, 'new_label'] = '1'
    elif sp == -1:
        data.at[index, 'new_label'] = '-1'
    else:
        data.at[index, 'new_label'] = '0'

    # mecab.pos 함수를 호출하기 전에 body가 str 타입인지 확인
    if isinstance(row['Review'], str):
        tokenized_comments = []
        pos_tags = mecab.pos(row['Review'])
        for token, pos in pos_tags:
            # 형태소가 동사(VV) 또는 형용사(VA)인 경우에만 tokenized_comments에 추가
            if pos in ['Noun', 'Adjective']:
                tokenized_comments.append(token)
        data.at[index, 'tokenized_comments'] = tokenized_comments

1/1 [==============================] - 1s 628ms/step
99.98% 확률로 부정 리뷰입니다.
1/1 [==============================] - 0s 40ms/step
62.44% 확률로 긍정 리뷰입니다.


C:\Users\USER\AppData\Local\Temp\ipykernel_13632\2931568357.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  score = float(score)  # 배열에서 스칼라 값으로 변환


1/1 [==============================] - 0s 54ms/step
99.09% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 41ms/step
92.45% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 52ms/step
82.66% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 53ms/step
99.24% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 45ms/step
89.87% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 37ms/step
98.43% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 32ms/step
93.54% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 43ms/step
96.85% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 37ms/step
74.78% 확률로 부정 리뷰입니다.
1/1 [==============================] - 0s 93ms/step
94.25% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 38ms/step
99.44% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 37ms/step
99.51% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 36ms/step
82.93% 확률로 긍정 리뷰입니다.
1/1 [==============================] - 0s 36ms/step

In [81]:
# 토크나이저의 단어 인덱스 확인
print(tokenizer.word_index)

# 모델의 임베딩 층의 가중치 가져오기
embedding_layer = loaded_model.layers[0]  # 임베딩 층의 인덱스는 모델의 첫 번째 층일 가능성이 높습니다.
embedding_weights = embedding_layer.get_weights()[0]

# 임베딩 가중치의 크기 출력
print(embedding_weights.shape)  # (vocab_size, embedding_dim)

{'OOV': 1, '멋지': 2, '고맙': 3, '재밌': 4, '알차': 5, '조': 6, '밝': 7, '푸하': 8, '슴슴하': 9, '실하': 10, '한결같': 11, '변함없': 12, '갈리': 13, '다시': 14, '반하': 15, '찐하': 16, '삐': 17, '메': 18, '머': 19, '끝내주': 20, '졸': 21, '가늘': 22, '형편없': 23, '녀': 24, '만지': 25, '까다롭': 26, '취하': 27, '답': 28, '흔하': 29, '개': 30, '잊': 31, '조르': 32, '파': 33, '더하': 34, '소문나': 35, '갔다오': 36, '귀엽': 37, '무섭': 38, '놀래': 39, '걸맞': 40, '날카롭': 41, '뛰어나': 42, '아름답': 43, '낯설': 44, '옳': 45, '꾀': 46, '띄': 47, '떠오르': 48, '알려주': 49, '갖추': 50, '보여주': 51, '나뉘': 52, '놀러오': 53, '잘생기': 54, '헤': 55, '잡치': 56, '절': 57, '풀리': 58, '엄': 59, '뽑': 60, '긋': 61, '어쩧': 62, '순하': 63, '붉': 64, '쌩까': 65, '갈하': 66, '내려가': 67, '쓰리': 68, '도와주': 69, '제공받': 70, '문제없': 71, '미루': 72, '드물': 73, '어처구니없': 74, '고프': 75, '진행하': 76, '잘': 77, '흥겹': 78, '별다르': 79, '어이없': 80, '괜챦': 81, '떨어트리': 82, '비우': 83, '살리': 84, '감': 85, '흘르': 86, '꼬': 87, '묵': 88, '붓': 89, '잊어버리': 90, '뵈': 91, '해먹': 92, '싸이': 93, '오지': 94, '끓': 95, '다지': 96, '깍': 97, '꺼지': 98, '접하': 99, '반기': 100, '헤매'

In [20]:
data.to_csv('./data/data_vv.csv', encoding='utf-8-sig')

In [4]:
data.loc[data['label'] == -1, 'label'] = 0

In [88]:
data

,Unnamed: 0,review_id,body,visited,viewCount,restaurant_id,businessName,rating,nouns,verbs,adjectives,exclamations,label,tokenized,new_label,tokenized_comment,tokenized_comments
0,33,60d00802ddab3800506106eb,손님 놓치기 싫어서 지하층으로 손님을 받을거면 동일한 서비스를 제공하세요 그동안 ...,21.6.21.월,494,11487063,구이마마 대치점,0.5,"['손님', '지하층', '손님', '거', '동일', '서비스', '제공', '앞...","['놓치', '싫', '받', '이', '가', '갈', '이', '하', '달라'...","['싫', '없']","['기', '어서', '을', '면', 'ㄴ', '세요', '었', '지만', 'ㅂ...",0,"['놓치', '싫', '받', '이', '가', '갈', '이', '하', '달라'...",1,None,"[놓치, 싫, 받, 없, 눈치보, 되, 깎, 받, 하, 수근거리]"
1,34,60b63d1a74dab90050fa6896,김치찌개랑 계란말이 쨩,21.5.28.금,453,11487063,구이마마 대치점,5.0,"['김치찌개', '계란', '말']",[],[],[],1,[],0,None,[]
2,35,60b434324caa1a005051f04e,파절이가 좀 아쉽네요,21.5.26.수,384,11487063,구이마마 대치점,3.5,['파절'],"['가', '아쉽']",['아쉽'],"['아', '네요']",1,"['가', '아쉽', '아쉽']",0,None,[아쉽]
3,36,607edb15268a2a004fd62779,맛있어요,21.4.13.화,311,11487063,구이마마 대치점,5.0,[],['맛있'],['맛있'],['어요'],1,"['맛있', '맛있']",0,None,[맛있]
4,37,6061b37ac701b50075941464,바빠요,21.3.29.월,300,11487063,구이마마 대치점,4.0,[],['바쁘'],['바쁘'],['아요'],1,"['바쁘', '바쁘']",0,None,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11697,30007,5d0b71e5484d1eee9d932761,케익이 맛있는 아티제,19.6.20.목,10,13066017,아티제 삼성타운점,4.0,"['케익', '티']","['맛있', '아']",['맛있'],"['는', '아']",1,"['맛있', '아', '맛있']",0,None,[맛있]
11698,30008,5ca6ef601b2052a5d8f567cd,삼성타운에 맛집이 참 많네요 그래도 커피 케익은 아티재 인가봅니다 페이코 할인되는 ...,19.4.5.금,16,13066017,아티제 삼성타운점,3.0,"['삼성', '타운', '맛', '집', '커피', '케익', '코', '할인', ...","['많', '그리하', '이', '보', '꾸']",['많'],"['네요', '여도', 'ㄴ가', 'ㅂ니다', '는', 'ㄹ']",0,"['많', '그리하', '이', '보', '꾸', '많']",0,None,[많]
11699,30009,5c977ab3a676ed62866c886f,주차가돼요,19.3.24.일,88,13066017,아티제 삼성타운점,5.0,['주차'],['되'],[],['어요'],1,['되'],0,None,[]
11700,30010,5c7fb58af6aecd8a0acd4eb8,깔끔하고 저녁엔 회사 사람들이 빠져나가서 조용하고 좋네요,19.3.6.수,10,13066017,아티제 삼성타운점,4.0,"['저녁', '회사', '사람']","['빠져나가', '좋']",['좋'],"['고', '서', '고', '네요']",1,"['빠져나가', '좋', '좋']",0,None,"[빠져나가, 좋]"


In [11]:
data['Rating'].dtype

dtype('int64')

In [12]:
data['new_label'] = data['new_label'].astype('int32')

In [13]:
data['new_label'].dtype

dtype('int32')

In [17]:
same_count = (data['Rating'] == data['new_label']).sum()
different_count = len(data) - same_count

# 결과 출력
print(f"같은 것의 개수: {same_count}")
print(f"다른 것의 개수: {different_count}")

같은 것의 개수: 9277
다른 것의 개수: 1209


In [18]:
# TN, TP, FN, FP 계산
TN = ((data['Rating'] == 0) & (data['new_label'] == 0)).sum()
TP = ((data['Rating'] == 1) & (data['new_label'] == 1)).sum()
FN = ((data['Rating'] == 1) & (data['new_label'] == 0)).sum()
FP = ((data['Rating'] == 0) & (data['new_label'] == 1)).sum()

# 결과 출력
print(f"TN: {TN}")
print(f"TP: {TP}")
print(f"FN: {FN}")
print(f"FP: {FP}")

TN: 427
TP: 8850
FN: 1118
FP: 91


In [19]:
# 전체 샘플 수
total_samples = len(data)

# 정확하게 예측된 샘플 수
correct_predictions = TP + TN

# 정확도 계산
accuracy = correct_predictions / total_samples

# 결과 출력
print(f"정확도: {accuracy}")

정확도: 0.8847034140759107


In [26]:
data.to_csv("data.csv", encoding='utf-8-sig')

In [76]:
test_data.dropna(subset="rating")

,review_id,body,visited,viewCount,restaurant_id,businessName,rating,tokenized,label,new_label,tokenized_comment,tokenized_comments
33,60d00802ddab3800506106eb,손님 놓치기 싫어서 지하층으로 손님을 받을거면 동일한 서비스를 제공하세요 그동안 ...,21.6.21.월,494,11487063,구이마마 대치점,0.5,"['손님', '놓치', '기', '싫', '어서', '지하층', '으로', '손님'...",부정,0,None,"[손, 지하, 층, 손님, 거, 서비스, 제공, 동안, 앞, 번, 사장, 남자, 분..."
34,60b63d1a74dab90050fa6896,김치찌개랑 계란말이 쨩,21.5.28.금,453,11487063,구이마마 대치점,5.0,"['김치찌개', '랑', '계란', '말', '쨩']",긍정,1,None,"[김치찌개, 계란, 말]"
35,60b434324caa1a005051f04e,파절이가 좀 아쉽네요,21.5.26.수,384,11487063,구이마마 대치점,3.5,"['파절', '아', '좀', '아쉽', '네요']",긍정,0,None,"[파절, 이]"
36,607edb15268a2a004fd62779,맛있어요,21.4.13.화,311,11487063,구이마마 대치점,5.0,"['맛있', '어요']",긍정,0,None,[]
37,6061b37ac701b50075941464,바빠요,21.3.29.월,300,11487063,구이마마 대치점,4.0,"['바쁘', '아요']",긍정,0,None,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
30007,5d0b71e5484d1eee9d932761,케익이 맛있는 아티제,19.6.20.목,10,13066017,아티제 삼성타운점,4.0,"['케익', '맛있', '아', '아', '티', '제']",긍정,0,None,"[케익, 티]"
30008,5ca6ef601b2052a5d8f567cd,삼성타운에 맛집이 참 많네요 그래도 커피 케익은 아티재 인가봅니다 페이코 할인되는 ...,19.4.5.금,16,13066017,아티제 삼성타운점,3.0,"['삼성', '타운', '맛', '집', '참', '많', '네요', '그리하', ...",부정,0,None,"[삼성타운, 맛집, 커피, 케익, 아티, 페이, 코, 할인, 건, 팁]"
30009,5c977ab3a676ed62866c886f,주차가돼요,19.3.24.일,88,13066017,아티제 삼성타운점,5.0,"['주차', '되', '어요']",긍정,0,None,[주차]
30010,5c7fb58af6aecd8a0acd4eb8,깔끔하고 저녁엔 회사 사람들이 빠져나가서 조용하고 좋네요,19.3.6.수,10,13066017,아티제 삼성타운점,4.0,"['깔끔', '저녁', '회사', '사람', '빠져나가', '서', '조용', '좋...",긍정,0,None,"[저녁, 회사, 사람]"


In [25]:
test_data.loc[data['label'] == '긍정', 'label'] = 1
test_data.loc[data['label'] == '부정', 'label'] = 0

NameError: name 'test_data' is not defined